# Recherche tabu

In [200]:
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

from collections import deque
import random
from functools import lru_cache
from IPython.display import clear_output


from functions import get_vrptw_instance, show_routes

## Variables

In [201]:
# Instances
instance_name = 'C101.txt'
data_set, data_set_best_solution = get_vrptw_instance(instance_name)

# DataSET
capacity = data_set['capacity']
coords = data_set['node_coord']
demand = data_set['demand']
nb_client = len(coords) - 1

# dataSetSolution
instance_best_solution = data_set_best_solution['routes']

## Mathematics things

In [202]:
# Distance between two points
points = np.array(coords)

DISTANCE = distance_matrix(points, points)

print(DISTANCE[1, 0])

18.681541692269406


## Giant tour manipulation

In [203]:
def merge(solution):
    """
    Merge a solution into a giant list
    Args:
        solution(list[list[int]]):
            A solution of the vrp.
            With k lists of c int elements.
            With k number of routes(trucks) and c number of client in the route.
    Returns:
        giant_tour(list[int]): Merge of the all routes into a giant list.
    """
    giant_tour = []
    for route in solution:
        for city in route:
            if city != 0:
                giant_tour.append(city)
    return giant_tour



In [204]:
def split(giant_tour):
    """
    Split the giant tour into multiple feasible routes.
    Args:
        giant_tour (list[int]): A giant tour representing a full permutation
        of all clients.

    Returns:
        sol (list[list[int]]): A list of routes, where each route is a list
        of clients assigned to the same truck, respecting capacity constraints.
    """
    route=[]
    sol =[]
    cost =0 
    for client in giant_tour:
        if cost + demand[client] <= capacity:
            route.append(client)
            cost += demand[client]
        else :
            sol.append(route)
            cost = demand[client]
            route = [client] 
    sol.append(route)

    return sol 

## random solution

In [205]:
def random_solution(nb_client):
    """
    Generate a random VRP solution.
    Args:
        None

    Returns:
        solution (list[list[int]]): A feasible solution composed of several routes,
        obtained by generating a random giant tour and splitting it into routes
        respecting the truck capacity constraints.
    """

    giant_tour = random.sample(range(nb_client + 1), k = nb_client + 1)
    solution = split(giant_tour)
    return solution

## Neighborhood

In [206]:
def neighborhood(solution, nb_neighbors, nb_swaps_per_neighbor):
    """
    Find nb_neighbors solution's neighbor
    Args:
        solution(list[list[int]]):
            A solution of the vrp.
            With k lists of c int elements.
            With k number of routes(trucks) and c number of client in the route.
        nb_neighbors(int):
            number of neighbors that we want to have for the solution
        nb_swaps_per_neighbor(int):
            number of swap per neighbor.
            if nb_swaps_per_neighbor = 1 we swap 2 values between them
            if nb_swaps_per_neighbor = 2 we swap two time 2 differente values between them
    Returns:
        neighbors(list[solution]):
            list of nb_neighbors solution's neighbor
    """
    neighbors = []

    giant_tour = merge(solution)
    size_giant_tour = len(giant_tour)

    i, j = random.sample(range(size_giant_tour), 2)
    for _ in range(nb_neighbors):
        neighbor_giant_tour = giant_tour.copy()
        for _ in range(nb_swaps_per_neighbor):
            i, j = random.sample(range(size_giant_tour), 2)
            neighbor_giant_tour[i], neighbor_giant_tour[j] = neighbor_giant_tour[j], neighbor_giant_tour[i]
        neighbors.append(split(neighbor_giant_tour))
    return neighbors

## Weights evaluation

In [207]:
def total_travel_distance(solution):
    """
    Compute the total distance travelled by all trucks.

    Args:
        solution (list[list[int]]): Set of routes, where each route lists the clients
            visited by a truck, without the depot.

    Returns:
        float: Sum of the distances for every leg of every route, including depot
            departure and return.
    """
    total_distance = 0
    for route in solution:
        full_route = [0] + route + [0]
        for j in range(len(full_route) - 1):
            origin = full_route[j]
            destination = full_route[j + 1]
            total_distance += DISTANCE[origin, destination]
    return total_distance

## Tabu search

In [ ]:
def tabu_search(initial_solution, tabu_size, iter_max, nb_neighbors, nb_swaps_per_neighbor, speed_graphic=0, show_graphic=False):
    """
    Perform a Tabu Search to improve a VRP solution.

    Args:
        initial_solution (list[list[int]]): Starting set of routes.
        tabu_size (int): Maximum number of solutions remembered in the tabu list.
        iter_max (int): Number of iterations allowed without improvement before stop.

    Returns:
        list[list[int]]: Best solution found during the search.
    """
    nb_iter = 0
    tabu_list = deque(maxlen=tabu_size)

    current_solution = initial_solution
    neighbor_best_solution = initial_solution
    global_best_solution = initial_solution

    value_best_neighbor = total_travel_distance(initial_solution)
    value_best_global = value_best_neighbor

    graphic_tic = 0
    while nb_iter < iter_max:
        for neighbor in neighborhood(current_solution, nb_neighbors, nb_swaps_per_neighbor):
            if total_travel_distance(neighbor) < value_best_neighbor and neighbor not in tabu_list:
                value_best_neighbor = total_travel_distance(neighbor)
                neighbor_best_solution = neighbor

        if value_best_neighbor < value_best_global:
            global_best_solution = neighbor_best_solution
            value_best_global = value_best_neighbor
            nb_iter = 0
        else:
            nb_iter += 1

        # Graph display
        if show_graphic and graphic_tic >= speed_graphic:
            clear_output(wait=True)
            print(f"Iteration: {nb_iter} | Best Distance: {value_best_global:.2f}")
            show_routes(current_solution, data_set)
            plt.close()
            graphic_tic = 0
        
        if show_graphic:
            graphic_tic += 1

        current_solution = neighbor_best_solution
        tabu_list.append(current_solution)
  
    return global_best_solution

In [209]:
# importantes variables for tabu search
tabu_size = nb_client
nb_iter = 200
nb_neighbors = 50
nb_swaps_per_neighbor = 1

best_score = float('inf')
best_solution = None

random.seed(3)

# Tabu search with multi start
for _ in range(10):
    cur = random_solution(nb_client)
    cur = tabu_search(cur, tabu_size, nb_iter, nb_neighbors, nb_swaps_per_neighbor)
    cur_score = total_travel_distance(cur)
    if cur_score < best_score:
        best_score = cur_score
        best_solution = cur

distance_best = total_travel_distance(instance_best_solution)
distance_tabu = total_travel_distance(best_solution)

print(f"Upper bound: {distance_best}")
print(f"Tabu best: {distance_tabu}")

gap = (distance_tabu - distance_best) / distance_best * 100
print(f"Gap: {gap:.2f}%")



Upper bound: 828.9368669428343
Tabu best: 1271.1077074255604
Gap: 53.34%
